In [1]:
import pandas as pd
import numpy as np
import re
import json
import os

from src.data_completion import *
from src.data_preprocessing import *
from src.data_loading import *
from src.data_fetching import *
from src.data_visualization import *

import statsmodels.formula.api as smf

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go


%load_ext autoreload
%autoreload 2

In [2]:

# # Example: Create a simple scatter plot
# df = px.data.iris()  # Sample dataset
# fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species", title="Sepal Dimensions")
# fig.show()
# # Save the plot as an HTML file
# fig.write_html("plot.html")

In [2]:
# Load the preprocessed datasets
full_movie_data_preprocessed = pd.read_csv('data/preprocessed/full_movie_data_preprocessed.csv')
full_characters_data_preprocessed = pd.read_csv('data/preprocessed/full_characters_data_preprocessed.csv')
subset_movie_with_full_data_on_lead_actors = pd.read_csv('data/preprocessed/subset_movie_with_full_data_on_lead_actors.csv')
lead_actors_data_on_subset_movie = pd.read_csv('data/preprocessed/lead_actors_data_on_subset_movie.csv')
characters_data_on_subset_movie = pd.read_csv('data/preprocessed/characters_data_on_subset_movie.csv')

# We must convert dates to datetime
lead_actors_data_on_subset_movie['actor_dob'] = pd.to_datetime(lead_actors_data_on_subset_movie['actor_dob'])
characters_data_on_subset_movie['actor_dob'] = pd.to_datetime(characters_data_on_subset_movie['actor_dob'])
full_movie_data_preprocessed['movie_release_date'] = pd.to_datetime(full_movie_data_preprocessed['movie_release_date'])


In [3]:
movie_data_completed = full_movie_data_preprocessed.copy()
movie_data_completed.describe()

,wikipedia_movie_id,movie_release_date,box_office_revenue,runtime,averageRating,numVotes,release_year,adjusted_box_office
count,3.493000e+03,3493,3.493000e+03,3493.000000,3493.000000,3.493000e+03,3493.000000,3.493000e+03
mean,7.225853e+06,1995-05-02 01:52:57.440595456,4.167977e+07,107.578214,6.268394,6.065138e+04,1994.808188,9.074958e+07
min,3.746000e+03,1958-01-29 00:00:00,1.000000e+00,61.000000,1.700000,5.060000e+02,1958.000000,1.913719e+00
25%,1.076567e+06,1987-10-11 00:00:00,4.106588e+06,95.000000,5.700000,5.905000e+03,1987.000000,9.125168e+06
50%,3.058252e+06,1996-10-11 00:00:00,1.471507e+07,104.000000,6.400000,1.850700e+04,1996.000000,3.238625e+07
75%,9.932614e+06,2005-06-10 00:00:00,4.100000e+07,116.000000,6.900000,5.922600e+04,2005.000000,9.459313e+07
max,3.681425e+07,2012-11-02 00:00:00,9.634204e+08,198.000000,8.800000,2.318314e+06,2012.000000,4.022758e+09
std,8.926649e+06,NaN,7.977446e+07,18.176518,0.972713,1.300172e+05,11.856645,1.806332e+08


In [5]:

def plotly_histogram(
    data: pd.DataFrame,
    columns: list,
    titles: list,
    labels: list,
    bins=50,
    log_scale=False,
    kdes=False,
    hue=None,
):
    """Plot histograms for the specified columns using Plotly.

    Args:
        data (pd.DataFrame): The dataset to plot.
        columns (list): The columns from the dataset to plot.
        titles (list): The titles for each plot.
        labels (list): The x labels for each plot.
        bins (int or list, optional): The number of bins. Defaults to 50.
        log_scale (bool or list, optional): If true, uses a log scale. Defaults to False.
        kdes (bool or list, optional): If true, adds a kde curve. Defaults to True.
        hue (str or None, optional): Name of the column in the dataset for color grouping. Defaults to None.
    """
    if not isinstance(bins, list):
        bins = [bins] * len(columns)

    if not isinstance(log_scale, list):
        log_scale = [log_scale] * len(columns)

    if not isinstance(kdes, list):
        kdes = [kdes] * len(columns)

    figs = []
    for i, col in enumerate(columns):
        hist_fig = px.histogram(
            data,
            x=col,
            color=hue,
            nbins=bins[i],
            title=titles[i],
            labels={col: labels[i]},
            marginal="box" if kdes[i] else None,
        )
        if log_scale[i]:
            hist_fig.update_layout(
                xaxis=dict(type="log"),
                yaxis=dict(type="log" if hue is None else "linear"),
            )
        hist_fig.update_layout(
            width=500,
            height=500,
        )
        figs.append(hist_fig)

    return figs

def histo_acots_plotly(actors: pd.DataFrame, hue=None):
    """Plot histograms for actor data using Plotly.

    Args:
    actors (pd.DataFrame): The actors dataset 
    hue (str or None): Name of the column in the dataset for color grouping. Defaults to None.
    """
    columns = ["actor_height", "actor_age_at_release", "actor_dob"]
    titles = [
        "Height of the actor",
        "Age of the actor at the release of the movie",
        "Date of birth of the actor",
    ]
    labels = ["Height (m)", "Age (years)", "Date of birth"]

    return plotly_histogram(actors, columns, titles, labels, bins=25, hue=hue)


def histo_movies_plotly(
    movies: pd.DataFrame,
    hue: str | None = None,
    axes: list[plt.Axes] = None,
):
    """Plot histograms for the movie data.

    Args:
        movies (pd.DataFrame): The movie dataset.
        hue (str | None, optional): Name of the column in the dataset. Defaults to None.
        axes (list[plt.Axes], optional): The axes to plot on. Defaults to None.
    """
    columns = ["runtime", "box_office_revenue", "movie_release_date"]
    titles = [
        "Runtime of the movie",
        "Box office revenue of the movie",
        "Release date of the movie",
    ]
    labels = ["Runtime (min)", "Box office revenue (dollars, log scale)", "Release date"]
    log_scale = [False, True, False]

    return plotly_histogram(
        movies,
        columns,
        titles,
        labels,
        bins=50,
        log_scale=log_scale,
        hue=hue
    )


In [18]:
# Updated version for histogram movies with subplot + logscale
movie_data_completed["movie_release_year"] = pd.to_datetime(movie_data_completed["movie_release_date"]).dt.year
columns = ["runtime", "box_office_revenue", "movie_release_year"]
titles = [
        "Runtime of the movie",
        "Box office revenue of the movie",
        "Release date of the movie",
]
labels = ["Runtime (min)", "Box office revenue (dollars, log scale)", "Release date"]
#log_scale = [False, True, False]

fig = make_subplots(rows=2, cols=2, subplot_titles=titles)

fig.add_trace(
    go.Histogram(x=movie_data_completed[columns[0]], 
                 nbinsx=50, histnorm='probability'), 
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=np.log10(movie_data_completed[columns[1]]), histnorm='probability', nbinsx=50),
    row=1, col=2
)

fig.add_trace(
    go.Histogram(x=movie_data_completed[columns[2]], histnorm='probability', nbinsx=50),
    row=2, col=1
)
# Customize the x-axis ticks for the first subplot
fig.update_xaxes(
    tickmode='linear',  # Use linear tick mode
    tick0=0,            # Start the ticks at 0
    dtick=20,           # Set the tick step to 20
    range=[60, 200],     # Limit the range of the x-axis to [0, 200]
    title_text=labels[0],
    row=1, col=1
)
fig.update_yaxes(title_text = 'Density', row=1, col=1)
fig.update_xaxes(
    tickvals=np.log10([10, 1e3, 1e5, 1e7, 1e9]),  # Log-transformed values
    ticktext=["10", "1K", "100K", "10M", "1B"],  # Original scale labels
    title_text=labels[1],
    row=1, col=2
)
fig.update_yaxes(title_text = 'Density', row=1, col=2)
fig.update_xaxes(
    tickmode='linear',      # Linear ticks for years
    dtick=10,                # Ticks every 5 years
    range=[1950, 2020],     # Set the range explicitly (as plain years)
    #tickformat=".0f",       # Force plain year formatting (e.g., 1980, 1985, ...)
    title_text=labels[2],
    row=2, col=1
)
fig.update_yaxes(title_text = 'Density', row=2, col=1)

fig.update_layout(height=700, width=800, showlegend=False, title_text="Numerical features of the movies")


# Show the figure
fig.show()
fig.write_html("movies_numeric_features.html")

In [8]:
#Demographic characteristics of lead actors 
figs_actor = histo_acots_plotly(lead_actors_data_on_subset_movie)
for i in figs_actor:
    i.show()
figs_actor[0].write_html("height_actor.html")
figs_actor[1].write_html("age_actor.html")


In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def count_plots(
    data,
    columns,
    titles,
    labels,
    cutoffs=None,
    horizontal=True,
    transforms=None,
    hue=None,
    axes=None,
):
    if axes is None:
        axes = []

    if not isinstance(cutoffs, list):
        cutoffs = [cutoffs] * len(columns)

    if not isinstance(transforms, list):
        transforms = [transforms] * len(columns)

    if not isinstance(horizontal, list):
        horizontal = [horizontal] * len(columns)

    for i, col in enumerate(columns):
        transform = transforms[i]
        if transform:
            col_data = transform(data[col])
            data_copy = pd.merge(
                col_data, data, left_index=True, right_index=True, suffixes=("_x", "")
            )
            col = f"{col}_x"
        else:
            data_copy = data

        col_counts = data_copy[col].value_counts().reset_index()
        col_counts.columns = [col, 'count']

        cutoff = cutoffs[i]
        if cutoff:
            col_counts = col_counts[:cutoff]

        fig = None
        if horizontal[i]:
            fig = px.bar(
                col_counts,
                y=col,
                x='count',
                orientation="h",
                title=titles[i],
                labels={col: labels[i], 'count': 'Count'},  
            )

        else:
            fig = px.bar(
                col_counts,
                x=col,
                y='count', 
                title=titles[i],
                labels={col: labels[i], 'count': 'Count'},
                
            )
        #uncomment for the 2nd plot, comment for 1st plot. opposite for the other update layout
        #fig.update_layout(title_font_size=16,title_x=0.5,
        #height=len(col_counts) * 10 + 100, 
        #yaxis=dict(tickmode="linear", tickfont=dict(size=10)),
        #margin=dict(l=120, r=10, t=50, b=50)  )
        fig.update_layout(title_font_size=16, title_x=0.5)
        axes.append(fig)
   

    return axes

def count_actors(
    actors,
    hue=None,
    axes=None,
):
    columns = ["actor_gender", "actor_ethnicity_label"]
    titles = ["Actor gender distribution", "Most common ethnicities"]
    labels = ["Gender", "Ethnicity"]
    cutoffs = [None, 20]
    horizontal = [False, True]

    return count_plots(
        actors,
        columns,
        titles,
        labels,
        cutoffs,
        horizontal=horizontal,
        hue=hue,
        axes=axes,
    )


figures = count_actors(lead_actors_data_on_subset_movie)
for fig in figures:
     fig.show()

#figures[1].write_html("actor_ethnicity.html")
figures[0].write_html("actor_gender.html")

#c'est normal que le graphe 1 soit casser quand on plot le graphe 2 avec tous les noms des ethnies tkt

In [10]:
figs_actor = histo_acots_plotly(lead_actors_data_on_subset_movie)
figs_all_actor = histo_acots_plotly(characters_data_on_subset_movie)

overlayed_figs = []

colors_actor = "blue"  
colors_all_actor = "rgba(255, 165, 0, 0.3)"  

for i in range(len(figs_actor)):
    fig = figs_actor[i]  

    for trace in fig.data:
        trace.marker.color = colors_actor
        trace.name = "Lead Actors"

    for trace in figs_all_actor[i].data:
        trace.marker.color = colors_all_actor
        trace.name = "All Actors"
        fig.add_trace(trace)  

    fig.update_layout(
        barmode="overlay",  
        legend_title="Dataset",
    )
    overlayed_figs.append(fig)

for i, fig in enumerate(overlayed_figs):
    fig.show()
    

In [11]:

def compare_gender_distribution(characters_data, lead_actors_data):
    gender_dist_all = characters_data['actor_gender'].value_counts(normalize=True) * 100
    gender_dist_lead = lead_actors_data['actor_gender'].value_counts(normalize=True) * 100

    labels = list(set(gender_dist_all.index).union(set(gender_dist_lead.index)))
    gender_dist_all = gender_dist_all.reindex(labels, fill_value=0)
    gender_dist_lead = gender_dist_lead.reindex(labels, fill_value=0)

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=labels,
        y=gender_dist_all,
        name='All actors',
        #marker=dict(color='blue'),
        offsetgroup=0  
    ))

    fig.add_trace(go.Bar(
        x=labels,
        y=gender_dist_lead,
        name='Lead actors',
        marker=dict(color='orange'),
        offsetgroup=1  
    ))

    fig.update_layout(
        title="Comparison of Actors' Gender Distribution Between Lead Roles and All Roles",
        xaxis_title="Gender",
        yaxis_title="Percent",
        barmode='group',
        bargap=0.2,
        bargroupgap=0.1,
        template='plotly_white'
    )
    fig.write_html("actor_lead_gender.html")
    fig.show()

compare_gender_distribution(characters_data_on_subset_movie, lead_actors_data_on_subset_movie)




In [12]:
def histogram_movie_ratings_plotly(
    movies: pd.DataFrame,
    hue: str | None = None,
):
    """Plot histograms for the movie ratings using Plotly.

    Args:
        movies (pd.DataFrame): The movie dataset.
        hue (str | None, optional): Name of the column in the dataset for color grouping. Defaults to None.
    """
    columns = ["averageRating", "numVotes"]
    titles = ["Average rating of the movie", "Number of votes for the movie"]
    labels = ["Average rating", "Number of votes (log scale)"]
    log_scale = [False, True]
    bins = [12, 50]
    kdes = [False, True]

    return plotly_histogram(
        movies,
        columns,
        titles,
        labels,
        bins=bins,
        log_scale=log_scale,
        kdes=kdes,
        hue=hue,
    )


In [13]:
figs = histogram_movie_ratings_plotly(movie_data_completed)
# Save the plot as an HTML file
# figs.write_html("hist_movies.html")
for i in figs:
    i.show()

In [14]:
def count_movies_plotly(
    movies: pd.DataFrame,
    hue: str | None = None,
):
    """Plot count plots for the movie data using Plotly.

    Args:
        movies (pd.DataFrame): The movie dataset.
        hue (str | None, optional): Name of the column in the dataset. Defaults to None.

    Returns:
        list[plotly.graph_objects.Figure]: List of plotly figures.
    """
    columns = ["genres", "languages", "countries"]
    titles = ["Most common genres", "Most common languages", "Most common countries"]
    labels = ["Genres", "Languages", "Countries"]
    cutoffs = [20, 10, 10]
    transforms = lambda col: col.apply(eval).explode()

    return count_plots_plotly(
        movies,
        columns,
        titles,
        labels,
        cutoffs,
        transforms=transforms,
        hue=hue,
    )

def count_plots_plotly(
    data: pd.DataFrame,
    columns: list[str],
    titles: list[str],
    labels: list[str],
    cutoffs: list[int] | int | None = None,
    transforms=None,
    hue: str | None = None,
):
    """Plot count plots for the specified columns using Plotly.

    Args:
        data (pd.DataFrame): The dataset to plot.
        columns (list[str]): The columns from the dataset to plot.
        titles (list[str]): The titles for each plot.
        labels (list[str]): The x labels for each plot.
        cutoffs (list[int] | int | None, optional): The cutoffs. Defaults to None.
        transforms ([type], optional): Transformation function for columns. Defaults to None.
        hue (str | None, optional): Name of the column in the dataset. Defaults to None.

    Returns:
        list[plotly.graph_objects.Figure]: List of plotly figures.
    """
    figures = []

    if not isinstance(cutoffs, list):
        cutoffs = [cutoffs] * len(columns)

    if not isinstance(transforms, list):
        transforms = [transforms] * len(columns)

    for i, col in enumerate(columns):
        transform = transforms[i]
        if transform:
            col_data = transform(data[col])
            col_data = col_data.value_counts().reset_index(name="count")
            col_data = col_data.rename(columns={"index": col})
        else:
            col_data = data[col].value_counts().reset_index(name="count")
            col_data = col_data.rename(columns={"index": col})

        cutoff = cutoffs[i]
        if cutoff:
            col_data = col_data.head(cutoff)

        col_data = col_data.sort_values(by="count", ascending=True)

        fig = px.bar(
            col_data,
            x="count",
            y=col,
            orientation="h",
            title=titles[i],
            labels={"count": "Count", col: labels[i]},
        )

        fig.update_layout(
            xaxis_title="Count",
            yaxis_title=labels[i],
            showlegend=False,
        )

        figures.append(fig)

    return figures


In [15]:
figs = count_movies_plotly(movie_data_completed)
for i in figs:
    i.show()
figs[0].write_html("genres_count.html")
figs[1].write_html("language_count.html")
figs[2].write_html("countries_count.html")